In [4]:
from utils.tobj import import_tobj
import meshio
import numpy as np
model = "boatv3"
V, T = import_tobj(f"assets/{model}/{model}.tobj")
mesh = meshio.Mesh(points = V, cells = [("tetra", T)])
mesh.write(f"assets/{model}/{model}.msh", file_format="gmsh22")

(18355, 3) (58415, 4)


Warning: Appending zeros to replace the missing physical tag data.

Warning: Appending zeros to replace the missing geometrical tag data.

In [3]:
obj_in = meshio.read(f"assets/{model}/{model}.obj")
off_out = meshio.write(f"assets/{model}/{model}.off", obj_in, file_format="off")

ReadError: File assets\boatv3\boatv3.obj not found.

In [5]:
import igl
F = igl.boundary_facets(T)
V, F, _, _ = igl.remove_unreferenced(V, F)
surface_mesh = meshio.Mesh(points = V, cells= [("triangle", F)])
surface_mesh.write(f"assets/{model}/{model}.obj")

In [6]:
surface_mesh.write(f"assets/{model}/{model}.off")

In [34]:
from pyqmat import qmat
from g2m.medial import SlabMesh
def compress_batch(task):
    model = "wheel"
    folder = f"assets/{model}/ma"
    full_model =  f"{folder}/full/{model}.ma"
    scaled_model = f"{folder}/scaled_{model}.ma"
    sm = SlabMesh(f"{full_model}", scale = 1e-3)
    sm.export_ma(scaled_model)
    for (model, nvs) in task.items():
        for nv in nvs:
            mat = qmat(f"assets/{model}/{model}.off", scaled_model)
            mat.simplify_slab(nv)
            mat.export_ply(f"{folder}/{model}_v{nv}")
            mat.export_ma(f"{folder}/{model}_v{nv}")

task_1 = {
    "wheel": [100, 80, 60, 40, 20]
}
compress_batch(task_1)